In [1]:
import numpy as np
import pandas as pd
from math import floor
from scipy.stats import norm

In [2]:
S_0 = 36
K = 40
T = 0.75
Sigma = 0.32
r = 0.04
q = 0.015

tau_final = T * (Sigma ** 2) / 2
x_compute = np.log(S_0 / K)
x_left = x_compute + (r - q - Sigma ** 2 / 2) * T - 3 * Sigma * np.sqrt(T)
x_right = x_compute + (r - q - Sigma ** 2 / 2) * T + 3 * Sigma * np.sqrt(T)

a = (r - q) / Sigma ** 2 - 1 / 2
b = (a + 1) ** 2 + 2 * q / Sigma ** 2

def f(x):
    return K * np.exp(a * x) * max(1 - np.exp(x), 0)

def g_left(tau):
    return K * np.exp(a * x_left + b * tau) * (np.exp(- 2 * r * tau / Sigma ** 2) - np.exp(x_left - 2 * q * tau / Sigma ** 2))

def g_right(tau):
    return 0

In [3]:
def grid_solver(M):
    alpha = 0.45
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    delta_x = (x_right - x_left) / N
    alpha = delta_T / delta_x ** 2
    grid = np.zeros((M+1, N+1), dtype=float)
    for i in range(M + 1):
        t_i = delta_T * i
        grid[i, 0] = g_left(t_i)
        grid[i, N] = g_right(t_i)

    for i in range(N + 1):
        x_i = x_left + i * delta_x
        grid[0, i] = f(x_i)

    for i in range(1, M + 1):
        for j in range(1, N):
            grid[i, j] = alpha * grid[i-1, j-1] + (1 - 2*alpha) * grid[i-1, j] + alpha * grid[i-1, j+1]
    
    return grid

def bsm_solver(S):
    d1 = (np.log(S / K) + (r - q + 0.5 * Sigma**2) * T) / (Sigma * np.sqrt(T))
    d2 = d1 - Sigma * np.sqrt(T)

    return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

output1 = np.round(grid_solver(4), 6)
# pd.DataFrame(output1).to_csv('./grid4.csv')

In [4]:
M_list = [4, 16, 64, 256]
col = ['e', 'ep', 'e2', 'ep2', 'er', 'erp', 'd', 'g', 't']
df = pd.DataFrame(index=M_list, columns=col)
V_exact = bsm_solver(S_0)

In [10]:
e = []
e2 = []
er = []
d = []
g = []
t = []

for M in M_list:
    alpha = 0.45
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    i = floor((x_compute - x_left) / np.sqrt(delta_T / alpha))
    grid = grid_solver(M)
    x = np.linspace(x_left, x_right, N+1)
    S = K * np.exp(x)
    u = grid[-1]

    V = np.exp(- a * x - b * tau_final) * u
    V_appro = ((S[i+1]-S_0) * V[i] + (S_0-S[i]) * V[i+1]) / (S[i+1] - S[i])
    V_error = abs(V_appro - V_exact)

    u_appro = ((x[i+1]-x_compute) * u[i] + (x_compute-x[i]) * u[i+1]) / (x[i+1] - x[i])
    V_appro2 = np.exp(-a*x_compute-b*tau_final) * u_appro
    V_error2 = abs(V_appro2 - V_exact)
    
    e.append(V_error)
    e2.append(V_error2)

    N_rms = 0
    v_rms = 0
    for k in range(N+1):
        flag = (V[k] > 0.00001 * S_0)
        if flag:
            N_rms += 1
            v_appro = np.exp(-a*x[k]-b*tau_final) * u[k]
            v_exact = bsm_solver(S[k])
            v_rms += ((v_appro - v_exact) / v_exact) ** 2
    e_rms = np.sqrt(v_rms / N_rms)
    er.append(e_rms)

    delta = (V[i+1]-V[i]) / (S[i+1]-S[i])
    gamma = ((V[i+2]-V[i+1])/(S[i+2]-S[i+1]) - (V[i]-V[i-1])/(S[i]-S[i-1])) / ((S[i+2]+S[i+1])/2 - (S[i]+S[i-1])/2)
    
    u2 = grid[-2]
    V2 = np.exp(- a * x - b * (tau_final - delta_T)) * u2
    V_appro_dt = ((S[i+1]-S_0) * V2[i] + (S_0-S[i]) * V2[i+1]) / (S[i+1] - S[i])
    dt = 2 * delta_T / Sigma ** 2
    theta = (V_appro_dt - V_appro) / dt

    d.append(delta)
    g.append(gamma)
    t.append(theta)

df['e'] = e
df['ep'] = df['e'] / df['e'].shift(1)
df['e2'] = e2
df['ep2'] = df['e2'] / df['e2'].shift(1)
df['er'] = er
df['erp'] = df['er'] / df['er'].shift(1)
df['d'] = d
df['g'] = g
df['t'] = t

In [11]:
df

,e,ep,e2,ep2,er,erp,d,g,t
4,0.210336,NaN,0.184461,NaN,0.098201,NaN,-0.582092,0.035349,-1.808942
16,0.050951,0.242237,0.045950,0.249106,0.060198,0.613001,-0.534090,0.037474,-1.846997
64,0.002604,0.051106,0.002411,0.052477,0.077182,1.282145,-0.537702,0.038376,-1.849582
256,0.003471,1.333032,0.003083,1.278353,0.093318,1.209069,-0.564394,0.038899,-1.841866


In [12]:
output2 = np.round(df, 6)
output2.to_csv('./grid.csv')

In [6]:
S_0 = 42
K = 40
T = 0.75
Sigma = 0.32
r = 0.04
q = 0.02

tau_final = T * (Sigma ** 2) / 2
x_compute = np.log(S_0 / K)
x_left = x_compute + (r - q - Sigma ** 2 / 2) * T - 3 * Sigma * np.sqrt(T)
x_right = x_compute + (r - q - Sigma ** 2 / 2) * T + 3 * Sigma * np.sqrt(T)

a = (r - q) / Sigma ** 2 - 1 / 2
b = (a + 1) ** 2 + 2 * q / Sigma ** 2

def f(x):
    return K * np.exp(a * x) * max(1 - np.exp(x), 0)

def g_left(tau):
    return K * np.exp(a * x_left + b * tau) * (1 - np.exp(x_left))

def g_right(tau):
    return 0

In [7]:
def grid_solver(M):
    alpha = 0.45
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    delta_x = (x_right - x_left) / N
    alpha = delta_T / delta_x ** 2
    
    early_grid = np.zeros((M+1, N+1), dtype=float)
    for m in range(1, M+1):
        for i in range(1, N):
            x = x_left + i * delta_x
            tau = m * delta_T
            early_grid[m, i] = K * np.exp(a * x + b * tau) * max(1 - np.exp(x), 0)

    grid = np.zeros((M+1, N+1), dtype=float)
    for i in range(M + 1):
        t_i = delta_T * i
        grid[i, 0] = g_left(t_i)
        grid[i, N] = g_right(t_i)

    for i in range(N + 1):
        x_i = x_left + i * delta_x
        grid[0, i] = f(x_i)

    for i in range(1, M + 1):
        for j in range(1, N):
            grid[i, j] = alpha * grid[i-1, j-1] + (1 - 2*alpha) * grid[i-1, j] + alpha * grid[i-1, j+1]
            grid[i, j] = max(grid[i, j], early_grid[i, j])
    
    return grid

output3 = np.round(grid_solver(4), 6)
output3

array([[28.295499, 23.462689, 18.452966, 13.230579,  7.75654 ,  1.988234,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ],
       [28.533931, 23.660398, 18.60846 , 13.342067,  7.8219  ,  3.576389,
         0.835334,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ],
       [28.774373, 23.859772, 18.765265, 13.454494,  8.357432,  4.20847 ,
         1.636002,  0.350956,  0.      ,  0.      ,  0.      ,  0.      ],
       [29.016841, 24.060827, 18.923391, 13.567868,  8.755766,  4.870816,
         2.176898,  0.743403,  0.14745 ,  0.      ,  0.      ,  0.      ],
       [29.261352, 24.263576, 19.082849, 13.79618 ,  9.145299,  5.371215,
         2.706451,  1.095287,  0.335884,  0.06195 ,  0.      ,  0.      ]])

In [8]:
M_list = [4, 16, 64, 256]
col = ['e', 'ep', 'e2', 'ep2', 'er', 'erp', 'd', 'g', 't']
df = pd.DataFrame(index=M_list, columns=col)
V_exact = 3.3045362802172642

In [11]:
e = []
e2 = []
er = []
d = []
g = []
t = []

for M in M_list:
    alpha = 0.45
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    i = floor((x_compute - x_left) / np.sqrt(delta_T / alpha))
    grid = grid_solver(M)
    x = np.linspace(x_left, x_right, N+1)
    S = K * np.exp(x)
    u = grid[-1]

    V = np.exp(- a * x - b * tau_final) * u
    V_appro = ((S[i+1]-S_0) * V[i] + (S_0-S[i]) * V[i+1]) / (S[i+1] - S[i])
    V_error = abs(V_appro - V_exact)

    u_appro = ((x[i+1]-x_compute) * u[i] + (x_compute-x[i]) * u[i+1]) / (x[i+1] - x[i])
    V_appro2 = np.exp(-a*x_compute-b*tau_final) * u_appro
    V_error2 = abs(V_appro2 - V_exact)
    
    e.append(V_error)
    e2.append(V_error2)

    # N_rms = 0
    # v_rms = 0
    # for k in range(N+1):
    #     flag = (V[k] > 0.00001 * S_0)
    #     if flag:
    #         N_rms += 1
    #         v_appro = np.exp(-a*x[k]-b*tau_final) * u[k]
    #         v_exact = bsm_solver(S[k])
    #         v_rms += ((v_appro - v_exact) / v_exact) ** 2
    # e_rms = np.sqrt(v_rms / N_rms)
    # er.append(e_rms)

    delta = (V[i+1]-V[i]) / (S[i+1]-S[i])
    gamma = ((V[i+2]-V[i+1])/(S[i+2]-S[i+1]) - (V[i]-V[i-1])/(S[i]-S[i-1])) / ((S[i+2]+S[i+1])/2 - (S[i]+S[i-1])/2)
    
    u2 = grid[-2]
    V2 = np.exp(- a * x - b * (tau_final - delta_T)) * u2
    V_appro_dt = ((S[i+1]-S_0) * V2[i] + (S_0-S[i]) * V2[i+1]) / (S[i+1] - S[i])
    dt = 2 * delta_T / Sigma ** 2
    theta = (V_appro_dt - V_appro) / dt

    d.append(delta)
    g.append(gamma)
    t.append(theta)

df['e'] = e
df['ep'] = df['e'] / df['e'].shift(1)
df['e2'] = e2
df['ep2'] = df['e2'] / df['e2'].shift(1)
# df['er'] = er
# df['erp'] = df['er'] / df['er'].shift(1)
df['d'] = d
df['g'] = g
df['t'] = t

In [12]:
df

,e,ep,e2,ep2,er,erp,d,g,t
4,0.270002,NaN,0.254607,NaN,NaN,NaN,-0.389221,0.030802,-2.579025
16,0.063669,0.235808,0.060425,0.237327,NaN,NaN,-0.339749,0.030656,-2.522116
64,0.009900,0.155489,0.009480,0.156891,NaN,NaN,-0.340174,0.031374,-2.524230
256,0.002786,0.281458,0.002624,0.276822,NaN,NaN,-0.365425,0.033041,-2.519832
